In [4]:
# import libraries
import pandas as pd
pd.set_option('display.max_colwidth', None) #setting max colwidth to view the entire dataset when using the print() command
import matplotlib.pyplot as plt
import numpy as np


from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier



from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from IPython.display import display

# import the files
data = open("./dataset/breast-cancer.data")
feat = open("./dataset/breast-cancer.names")

data = data.read()
feat = feat.read()

# replace missing dataset attributes to NAN
data = data.replace('?','')

from io import StringIO

# convert data from str to dataframe
data = StringIO(data)
data = pd.read_csv(data, sep=",")

data.columns = ['class', 'age', 'menopause', 'tumour_size', 'inv_nodes', 'node_caps', 'deg_malig', 'breast', 'breast_quad', 'irrad']



X = data.drop(['class'], axis = 1)
y = data['class']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

y_test = y_test.replace(['no-recurrence-events','recurrence-events'], [0,1])
y_train = y_train.replace(['no-recurrence-events', 'recurrence-events'], [0,1])


X_train.drop(['age','menopause', 'breast'], axis=1)
X_test.drop(['age','menopause', 'breast'], axis=1)



X_train = pd.get_dummies(X_train)


X_test = pd.get_dummies(X_test)
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)



In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2





from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score

from imblearn.over_sampling import RandomOverSampler

from IPython.display import display



def mix(X_train, X_test, y_train, y_test):
    k = 1
    while k < 50:
        X_trainin = X_train
        X_testin = X_test
        y_trainin = y_train
        y_testin = y_test
        selector = SelectKBest(chi2, k=k)
        X_train = selector.fit_transform(X_trainin, y_trainin)
        X_test = selector.transform(X_testin)

        clf = RandomForestClassifier()
        clf.fit(X_trainin, y_trainin)
        y_pred = clf.predict(X_testin)
        r_acc = clf.score(X_trainin, y_trainin)
        r_f1 = f1_score(y_testin, y_pred)
        print(r_acc)
        print(r_f1)
        k += 1

In [6]:
mix(X_train, y_train, X_test, y_test)

ValueError: Found input variables with inconsistent numbers of samples: [228, 57]